In [204]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import poisson as pois
from scipy.stats import chi2
import scipy.integrate as integrate
%matplotlib inline

# Завдання 1

Перевірка гіпотези однорідності.

   A. Критерій пустих блоків

In [155]:
def empty_blocks_criteria(n,m,alpha,beta):
    sel_1 = np.random.exponential(scale=1/alpha, size=n)
    sel_2 = np.random.exponential(scale=1/beta, size=m)
    sel_1_sorted = np.sort(sel_1)
    sel_2_min = np.amin(sel_2)
    sel_2_max = np.amax(sel_2)
    if (sel_2_min < sel_1_sorted[0]):
        bins = np.append(np.array(sel_2_min), sel_1_sorted)
    else:
        bins = np.append(np.array(sel_1_sorted[0]-0.1), sel_1_sorted)
    if (sel_2_max > sel_1_sorted[n-1]):
        bins = np.append(bins,np.array(sel_2_max))
    else:
        bins = np.append(bins,np.array(sel_1_sorted[n-1] + 0.1))
    inds = np.digitize(sel_2, bins)
    unique = np.arange(n+2) + 1
    frequencies = dict(((i,0) for i in unique))
    for i in inds:
        frequencies[i] += 1
    frequencies_list = np.fromiter(frequencies.values(), dtype=float)
    empty_blocks = 0
    for i in frequencies_list:
        if (i == 0):
            empty_blocks += 1
    gamma = 0.05
    z_gamma = norm.ppf(1-0.5*gamma, loc = 0, scale=1)
    ro = m/n
    k_star = n/(1+ro)+(ro*np.sqrt(n)*z_gamma/((1+ro) ** 1.5))
    print("empty blocks: {0}, k_star: {1}".format(empty_blocks, k_star))
    if(empty_blocks > k_star):
        print(False)
    else:
        print(True)

In [107]:
n = 500
m = 1000
alpha = 1
beta = 1.2

In [108]:
empty_blocks_criteria(n, m, alpha, beta)

empty blocks: 157, k_star: 183.53535082508878
True


In [109]:
n = 5000
m = 10000
alpha = 1
beta = 1.2

In [110]:
empty_blocks_criteria(n, m, alpha, beta)

empty blocks: 1656, k_star: 1720.0101297372812
True


   B. Критерій  $\chi^2$

In [212]:
def chi2_criteria(n1, n2, n3, r, lamda):
    sel_1 = np.sort(np.random.poisson(lam=lamda, size = n1))
    sel_2 = np.sort(np.random.poisson(lam=lamda, size = n2))    
    sel_3 = np.sort(np.random.poisson(lam=lamda, size = n3))
    bins = np.arange(r-1)+1
    flag = True
    unique = np.arange(r)
    while flag:
        flag_temp = True
        inds_1 = np.digitize(sel_1, bins)
        inds_2 = np.digitize(sel_2, bins)
        inds_3 = np.digitize(sel_3, bins)
        freq_1 = dict(((i,0) for i in unique))
        freq_2 = dict(((i,0) for i in unique))
        freq_3 = dict(((i,0) for i in unique))
        for i in inds_1:
            freq_1[i] += 1
        freq_1 = np.fromiter(freq_1.values(), dtype=int)
        for i in freq_1:
            if (i < 5):
                flag_temp = False
        for i in inds_2:
            freq_2[i] += 1
        freq_2 = np.fromiter(freq_2.values(), dtype=int)
        for i in freq_2:
            if (i < 5):
                flag_temp = False
        for i in inds_3:
            freq_3[i] += 1
        freq_3 = np.fromiter(freq_3.values(), dtype=int)
        for i in freq_3:
            if (i < 5):
                flag_temp = False
        if (flag_temp==True):
            flag = False
    print(bins)
    print(sel_1)
    print(freq_1)
    print(freq_2)
    print(freq_3)
    probs = np.empty(r)   
    probs[0] = pois.cdf(0, lamda);
    for i in np.arange(r-2)+1:
        probs[i] = pois.cdf(i, lamda) - pois.cdf(i-1, lamda);
    probs[r-1] = pois.cdf(120, lamda) - pois.cdf(r-2, lamda);
    print(probs*n1)
    delta = np.sum(np.divide((freq_1 - n1*probs)**2, n*probs))
    delta = delta + np.sum(np.divide((freq_2 - n2*probs)**2, n*probs))
    delta = delta + np.sum(np.divide((freq_3 - n3*probs)**2, n*probs))
    gamma = 0.05
    z_gamma = chi2.ppf(1 - 0.5*gamma, df = r-1)
    print(delta)
    if (delta>z_gamma):
        return False
    else:
        return True

In [213]:
chi2_criteria(200,500,800,5,2)

[1 2 3 4]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 5 5 5 5 5 5 5 6 6 7]
[36 45 46 42 31]
[ 52 155 148  75  70]
[115 209 231 135 110]
[27.06705665 54.13411329 54.13411329 36.08940886 28.5753079 ]
1.688659089946351


True

In [214]:
chi2_criteria(200,500,800,3,2)

[1 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 5 5 5 5 5 6 8]
[ 26  57 117]
[ 78 118 304]
[ 95 207 498]
[ 27.06705665  54.13411329 118.79883006]
0.9076363245195762


True